### ✅ Basic Syntax, Inputs, Conditionals, Enumerate And Loops

### ✅ Lists, Sets, Tuples, Strings, Dictionaries & Related

### ✅ Functions, Lambda and List Comprehensions

### ✅ Exceptions : Creating and Handling Them

### ✅ File I/O with Different Libraries & Modules

### ✅ Generators

In [1]:
def generate_numbers(num : int, power : int) -> int:
    for i in range(num):
        yield i**power

In [2]:
print(list(generate_numbers(5, 2)))

[0, 1, 4, 9, 16]


In [3]:
gen = generate_numbers(5, 2)
print(next(gen))
print(next(gen))
print(next(gen))

0
1
4


<h4> A More Detailed Example </h4>

* An elegant example where simple List comprehension would crash the RAM. 
* But, using generators in combination with multiprocessing queue saves the day
* In the below example, I am trying to find out the best possible threshold parameters for model output rounding

In [5]:
import numpy as np
import pandas as pd
from loguru import logger
from itertools import product
from sklearn.model_selection import train_test_split
from loguru import logger
import multiprocessing
import os
import traceback

##### Some clarity before going to the example
* Snippets similar to next cell will be used in code and things get confusing when generators are involved

In [10]:
dummies = [np.arange(i + 0.27, i + 0.90, 0.30).tolist() for i in range(2)] # I put 2 here because it is low number
print(f"Dummies -> {dummies}")
combinations = list(product(*dummies))
print(combinations) # see , it generates all the outputs because we used list() ; Imagine the number when we have range(12) instead of range(2)

Dummies -> [[0.27, 0.5700000000000001, 0.8700000000000001], [1.27, 1.57, 1.87]]
[(0.27, 1.27), (0.27, 1.57), (0.27, 1.87), (0.5700000000000001, 1.27), (0.5700000000000001, 1.57), (0.5700000000000001, 1.87), (0.8700000000000001, 1.27), (0.8700000000000001, 1.57), (0.8700000000000001, 1.87)]


In [ ]:
def within_2unit_diff(gt,pred):
    abs_diff = np.absolute(gt - pred)
    filler = np.zeros_like(abs_diff)
    class_indices = np.where(abs_diff <= 2)[0]
    filler[class_indices] = 1
    return np.sum(filler)/len(gt)


df = pd.read_csv("some path")
train, valid = train_test_split(df, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test  = train["pitch"], valid["pitch"],train["label_pred"], valid["label_pred"]

X_train = X_train.tolist()
X_test = X_test.tolist()
y_train = y_train.tolist()
y_test = y_test.tolist()

classes = set(X_train)
num_classes = len(classes) # say 12

dummies = [np.arange(i + 0.27, i + 0.90, 0.30).tolist() for i in range(num_classes)] # [0.27, 0.57, 0.87] are base values on which 'i' would get added to generate combinations

def generate_combinations(*lists):
    combinations = product(*lists) # gives all possible combinations
    for combo in combinations:
        yield combo

def generate_combinations_and_put(queue, *lists):
    combinations = list(generate_combinations(*lists))
    queue.put(combinations)

def get_scores(generator):
    best_score  = 0
    best_thresh = []
    counter = 0
    for combo in generator:
        counter += 1
        current_thresholds = combo
        bins = np.digitize(y_train, current_thresholds)
        score = within_2unit_diff(np.array(X_train), np.array(bins))
        if score > best_score:
            best_score = score
            best_thresh = current_thresholds
        if not counter%100:
            logger.debug(f"pid : {os.getpid()} ; processed : {counter} ; best_score : {best_score} ; best_thresh : {best_thresh}")

    return best_score, best_thresh

queue = multiprocessing.Queue()

generator_process = multiprocessing.Process(target=generate_combinations_and_put, args=(queue, *dummies))
generator_process.start()

combos = queue.get(timeout=60)
generator_process.join()

num_procs  = 50
pool = multiprocessing.Pool(processes=num_procs)
chunksize = len(combos) // num_procs

try:
    results = pool.map(get_scores, [combos[i:i+chunksize] for i in range(0, len(combos), chunksize)])
except Exception as e:
    traceback.print_exc()


logger.debug(results)
sc = [res[0] for res in results]
th = [res[1] for res in results]

sc_max_ix = np.argmax(np.array(sc))
logger.debug(sc[sc_max_ix])
logger.debug(th[sc_max_ix])

### ✅ Decorators

In [1]:
import datetime
from loguru import logger
import time
from functools import wraps

In [2]:
def f1():
    print("Called F1")

def f2(f):
    f()

print(f1) 
f2(f1) 

<function f1 at 0x7fbfdf6d9160>
Called F1


In [3]:
def f1(func):
    def wrapper():
        print("This is some work before the func")
        func()
        print("This is some work after the func")
    return wrapper

def f():
    print("hello")

x= f1(f)  # decorated f with f1
x()

This is some work before the func
hello
This is some work after the func


In [4]:
def f1(func):
    def wrapper():
        print("This is some work before the func")
        func()
        print("This is some work after the func")
    return wrapper
    
@f1
def f():
    print("hello")
f()

This is some work before the func
hello
This is some work after the func


In [6]:
def timer(function):
    @wraps(function) # optional here
    def wrapper():
        t1 = time.perf_counter()
        function()
        t2 = time.perf_counter()
        logger.debug(f"Time taken is {t2-t1} seconds...")
    return wrapper

@timer
def main():
    time.sleep(2)
    logger.info("Slept for 2 seconds...")

main()

2023-05-16 14:53:25.243 | INFO     | __main__:main:12 - Slept for 2 seconds...
2023-05-16 14:53:25.245 | DEBUG    | __main__:wrapper:6 - Time taken is 2.006161372999941 seconds...


In [13]:
def timer(func):
    def wrapper(*args, **kwargs):
        logger.info(args)
        if args[0] == 2:
            logger.info("Successful")
        else:
            logger.info("Failure")
        t1 = time.perf_counter()
        func(*args, **kwargs)
        t2 = time.perf_counter()
        logger.debug(f"Time taken is {t2-t1} seconds...")
    return wrapper

@timer
def main(sleep_duration, id = "3D4F-H5JJ-3SW1"):
    time.sleep(sleep_duration)
    logger.info(f"Slept for {sleep_duration} seconds with ID {id}")
    
main(2, id = "ROUND1")
logger.info("++++++++++++++++")
main(2, "ROUND2") 

2023-05-16 15:02:37.610 | INFO     | __main__:wrapper:3 - (2,)
2023-05-16 15:02:37.611 | INFO     | __main__:wrapper:5 - Successful
2023-05-16 15:02:39.617 | INFO     | __main__:main:17 - Slept for 2 seconds with ID ROUND1
2023-05-16 15:02:39.619 | DEBUG    | __main__:wrapper:11 - Time taken is 2.005651622999949 seconds...
2023-05-16 15:02:39.620 | INFO     | __main__:<module>:20 - ++++++++++++++++
2023-05-16 15:02:39.621 | INFO     | __main__:wrapper:3 - (2, 'ROUND2')
2023-05-16 15:02:39.621 | INFO     | __main__:wrapper:5 - Successful
2023-05-16 15:02:41.627 | INFO     | __main__:main:17 - Slept for 2 seconds with ID ROUND2
2023-05-16 15:02:41.628 | DEBUG    | __main__:wrapper:11 - Time taken is 2.0055156969999643 seconds...


In [17]:
def timer(func):
    def wrapper(*args, **kwargs):
        t1 = time.perf_counter()
        val = func(*args, **kwargs)
        t2 = time.perf_counter()
        logger.debug(f"Time taken is {t2-t1} seconds...")
        return val
    return wrapper

@timer
def main(sleep_duration, id = "3D4F-H5JJ-3SW1"):
    time.sleep(sleep_duration)
    logger.info(f"Slept for {sleep_duration} seconds with ID {id}")
    return 0

@timer   
def add(x,y):
    time.sleep(2)
    return x + y

add(2,3)
logger.debug("=========")
main(2)

2023-05-16 15:05:19.716 | DEBUG    | __main__:wrapper:6 - Time taken is 2.004138692999959 seconds...
2023-05-16 15:05:19.717 | DEBUG    | __main__:<module>:22 - =========
2023-05-16 15:05:21.718 | INFO     | __main__:main:13 - Slept for 2 seconds with ID 3D4F-H5JJ-3SW1
2023-05-16 15:05:21.720 | DEBUG    | __main__:wrapper:6 - Time taken is 2.0014227619999474 seconds...


0

In [39]:
import requests

def check_status_code(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        response = func(*args, **kwargs)  
        if response.status_code == 200:
            logger.debug(response.json())
        else:
            logger.error(f"Request failed with status code: {response.status_code}")
    return wrapper

def check_if_okay(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if kwargs.get("okay") != "YES":
            logger.debug({"status" : "Failed"})
        else:
            logger.debug(func(*args, **kwargs).json())
    return wrapper

@check_if_okay
def make_api_request(url, okay="YES"):
    logger.info("Sending Requests...")
    response = requests.get(url)
    return response

url = "https://api.github.com/"
make_api_request(url, okay="NO") # Output -> __main__:wrapper:17 - {'status': 'Failed'}

make_api_request(url, okay="YES")



2023-05-16 15:23:33.434 | DEBUG    | __main__:wrapper:17 - {'status': 'Failed'}
2023-05-16 15:23:33.436 | INFO     | __main__:make_api_request:24 - Sending Requests...
2023-05-16 15:23:33.945 | DEBUG    | __main__:wrapper:19 - {'current_user_url': 'https://api.github.com/user', 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}', 'authorizations_url': 'https://api.github.com/authorizations', 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}', 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}', 'emails_url': 'https://api.github.com/user/emails', 'emojis_url': 'https://api.github.com/emojis', 'events_url': 'https://api.github.com/events', 'feeds_url': 'https://api.github.com/feeds', 'followers_url': 'https://api.github.com/user/followers', 'following_url': 'https://api.github.com/user/following{/target}', 'gists_url': 'https://api.github.com/gist

### ✅ Functools

In [43]:
from functools import reduce

In [45]:
reduce(lambda x,y : x*y ,[2,3,4])

24

In [49]:
print(reduce(lambda x,y : max(x,y) , [1,2,3,4,5])) 
print(reduce(lambda x,y : max(x,y) , [1,2,3,4,5] , 10)) # 10 is the initializer ; hence (10,1) is the first pair

5
10


### ✅ Collections

In [50]:
from collections import Counter

In [51]:
a = "aaaabbbbsssaaadddd"
my_counter = Counter(a)

print(my_counter)

print(my_counter.keys()) 
print(my_counter.values())  
print(my_counter.most_common())
print(my_counter.most_common(2))



Counter({'a': 7, 'b': 4, 'd': 4, 's': 3})
dict_keys(['a', 'b', 's', 'd'])
dict_values([7, 4, 3, 4])
[('a', 7), ('b', 4), ('d', 4), ('s', 3)]
[('a', 7), ('b', 4)]


In [52]:
from collections import namedtuple

In [53]:
Point = namedtuple('Point', 'x,y')
pt = Point(1,-4)

print(pt)
print(pt.x , pt.y)

Point(x=1, y=-4)
1 -4


In [54]:
from collections import defaultdict

In [55]:
d = defaultdict(int)

d['a'] = 1
d['b'] = 2

print(d)

print(d['c'])  # see, we hadn't even assigned any val for 'c'

print(d)


defaultdict(<class 'int'>, {'a': 1, 'b': 2})
0
defaultdict(<class 'int'>, {'a': 1, 'b': 2, 'c': 0})


In [56]:
from collections import deque # double ended queue

In [57]:
d = deque()

d.append(1)
d.append(2)
d.append(2)
d.append(3)
print(d)

d.appendleft(-1)
print(d)

d.pop()
print(d)

d.popleft()
print(d)

d.extend([4,5,6])
print(d)

d.extendleft([0,0])
print(d)

d.rotate(1)
print(d)

d.rotate(2)
print(d)

d.rotate(-1)
print(d)

deque([1, 2, 2, 3])
deque([-1, 1, 2, 2, 3])
deque([-1, 1, 2, 2])
deque([1, 2, 2])
deque([1, 2, 2, 4, 5, 6])
deque([0, 0, 1, 2, 2, 4, 5, 6])
deque([6, 0, 0, 1, 2, 2, 4, 5])
deque([4, 5, 6, 0, 0, 1, 2, 2])
deque([5, 6, 0, 0, 1, 2, 2, 4])


### ✅ Heapq
* Implements min heap in Python
* Binary Heap Data Structure
* Helpful to implement Priority Queues

In [58]:
import heapq

In [59]:
heap = []
heapq.heappush(heap, 2)
heapq.heappush(heap, 3)
heapq.heappush(heap, 1)
heapq.heappush(heap, 6)
heapq.heappush(heap, -1)

In [60]:
heap

[-1, 1, 2, 6, 3]

In [61]:
a = heapq.heappop(heap)
print(a)
print(heap)

-1
[1, 3, 2, 6]


In [62]:
a = heapq.heappop(heap)
print(a)
print(heap)

1
[2, 3, 6]


In [63]:
## heapify -> convert the given list to binary heap

a = [2,4,-1,6,99,-128]
heapq.heapify(a) # inplace operation
print(a)

[-128, 4, -1, 6, 99, 2]


In [64]:
heapq.nsmallest(2, a)

[-128, -1]

In [65]:
heapq.nlargest(2, a)

[99, 6]